In [11]:
import os
from Network import QNetwork
from Agent import Agent
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import warnings
import grid2op
from grid2op.PlotGrid import PlotMatplot
from lightsim2grid import LightSimBackend
from grid2op.Action import TopologySetAction
from grid2op.gym_compat import DiscreteActSpace
from grid2op.gym_compat import GymEnv
import grid2op
from gymnasium import Env
from gymnasium.utils.env_checker import check_env
from Agent import Agent
from utils import obs_to_vect
from collections import deque, namedtuple

In [2]:
env_name = "rte_case5_example"  # or any other name.
env = grid2op.make(env_name, test=True, action_class=TopologySetAction)

c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\grid2op\MakeEnv\Make.py:438: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [4]:
try:
    from lightsim2grid import LightSimBackend
    bk_cls = LightSimBackend
except ImportError as exc:
    print(f"Error: {exc} when importing faster LightSimBackend")
    from grid2op.Backend import PandaPowerBackend
    bk_cls = PandaPowerBackend
    
env_name = "rte_case5_example"
training_env = grid2op.make(env_name, test=True)  # we put "test=True" in this notebook because...
# it's a notebook to explain things. Of course, do not put "test=True" if you really want
# to train an agent...
gym_env = GymEnv(training_env)

c:\Users\Ernest\.conda\envs\l2rpn-test\lib\site-packages\grid2op\MakeEnv\Make.py:438: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [5]:
gym_env.action_space = DiscreteActSpace(training_env.action_space)

In [9]:
HP = {}
HP['BUFFER_SIZE'] = int(1e5)
HP['BATCH_SIZE'] = 64
HP['GAMMA'] = 0.99
HP['TAU'] = 1e-3
HP['LR'] = 5e-4
HP['UPDATE_EVERY'] = 4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
agent = Agent(state_size=211, action_size=166, seed=1000, HP=HP, device=device)

In [14]:
def dqn(n_epsiode=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    for i_episode in range(1, n_epsiode+1):
        state, *_ = gym_env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(obs_to_vect(state))
            next_state, reward, done, _, _ = gym_env.step(action)
            agent.step(obs_to_vect(state), action, reward, obs_to_vect(next_state), done)
            state = next_state
            score += reward
            if done:
                break

            scores_window.append(score)
            scores.append(score)
            eps = max(eps_end, eps_decay*eps)
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
            if i_episode % 100 == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            if np.mean(scores_window)>=200.0:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
                torch.save(agent.qnet_local.state_dict(), 'checkpoint.pth')
                break
    return scores


In [15]:
scores = dqn()

Episode 1	Average Score: 201.63
Environment solved in -99 episodes!	Average Score: 201.63


AttributeError: 'Agent' object has no attribute 'qnetwork_local'